In [1]:
import os
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras import utils as np_utils

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [32]:
train_data_folder='Dataset/train/'
test_data_folder='Dataset/test/'
train_files=os.listdir(train_data_folder)

In [41]:
complete_data=None
string_to_int={}
int_to_string={}
counter=0

def fill_data(filename):
    data=np.load(file=train_data_folder+filename)
    filename=filename.split('.')[0]
    temp_var=-1
    
    global counter
    global complete_data
    global string_to_int
    global int_to_string
    
    if filename in string_to_int:
        temp_var=string_to_int[filename]
    
    else:
        temp_var=counter
        counter+=1
        string_to_int[filename]=temp_var
        int_to_string[temp_var]=filename
    
    dimension=data.shape
    temp_y=np.ones(shape=(dimension[0],1),dtype=np.float64)*temp_var
    data=np.concatenate((data,temp_y),axis=1)
    
    if complete_data is None:
        complete_data=np.copy(data)
    
    else:
        complete_data=np.concatenate((complete_data,data),axis=0)
    

for filename in train_files:
    fill_data(filename)

print(complete_data.shape)
print(string_to_int)
print(int_to_string)    

(100000, 785)
{'harp': 14, 'trombone': 16, 'bulldozer': 12, 'hand': 17, 'foot': 19, 'nose': 1, 'hat': 4, 'penguin': 13, 'chair': 6, 'parrot': 8, 'skyscraper': 0, 'pig': 15, 'snowman': 10, 'keyboard': 3, 'banana': 2, 'eyeglasses': 5, 'spider': 11, 'violin': 7, 'laptop': 18, 'flashlight': 9}
{0: 'skyscraper', 1: 'nose', 2: 'banana', 3: 'keyboard', 4: 'hat', 5: 'eyeglasses', 6: 'chair', 7: 'violin', 8: 'parrot', 9: 'flashlight', 10: 'snowman', 11: 'spider', 12: 'bulldozer', 13: 'penguin', 14: 'harp', 15: 'pig', 16: 'trombone', 17: 'hand', 18: 'laptop', 19: 'foot'}


In [42]:
complete_data=np.random.permutation(complete_data)
print(complete_data.shape)

(100000, 785)


In [43]:
[instances,dimensions]=complete_data.shape

x_train=complete_data[:,0:dimensions-1].astype(np.float32)
test_x=np.load(file="Dataset/test/test.npy").astype(np.float32)
y_train2=complete_data[:,dimensions-1:].astype(np.int32)
y_train = np_utils.to_categorical(y_train2)
print(x_train.shape)
print(y_train.shape)
print(y_train2.shape)
print(test_x.shape)
categories=20

(100000, 784)
(100000, 20)
(100000, 1)
(100000, 784)


In [44]:
# param=1000
# x_train=x_train[0:param,:]
# y_train=y_train[0:param]
# y_train.shape
x_train/=255.0
test_x/=255.0
x_train=x_train.reshape((100000,28,28,1))
test_x=test_x.reshape((100000,28,28,1))

In [12]:
model = Sequential()
model.add(Conv2D(32, (5, 5), padding='same', activation='relu', input_shape=(28,28,1)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.35))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# model.add(Dropout(0.35))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.35))
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(Dropout(0.35))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(categories, activation='softmax'))

In [45]:
batch_size = 4096
epochs = 500
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_split=0.1)
# model1.evaluate(test_data, test_labels_one_hot)

Train on 90000 samples, validate on 10000 samples
Epoch 1/500
90000/90000 [==============================] - 10s 108us/step - loss: 0.0712 - acc: 0.9788 - val_loss: 0.0072 - val_acc: 0.9982
Epoch 2/500
90000/90000 [==============================] - 9s 98us/step - loss: 0.0693 - acc: 0.9798 - val_loss: 0.0075 - val_acc: 0.9985
Epoch 3/500
90000/90000 [==============================] - 9s 100us/step - loss: 0.0718 - acc: 0.9790 - val_loss: 0.0113 - val_acc: 0.9967
Epoch 4/500
90000/90000 [==============================] - 9s 98us/step - loss: 0.0672 - acc: 0.9804 - val_loss: 0.0080 - val_acc: 0.9981
Epoch 5/500
90000/90000 [==============================] - 9s 98us/step - loss: 0.0675 - acc: 0.9801 - val_loss: 0.0079 - val_acc: 0.9982
Epoch 6/500
90000/90000 [==============================] - 9s 98us/step - loss: 0.0686 - acc: 0.9797 - val_loss: 0.0091 - val_acc: 0.9977
Epoch 7/500
90000/90000 [==============================] - 9s 98us/step - loss: 0.0681 - acc: 0.9799 - val_loss: 0.0083

KeyboardInterrupt: 

In [38]:
def label_to_string(label):
    lis=[]
    instances=label.shape[0]
    for i in range(instances):
        lis.append(int_to_string[label[i]])
    return lis

In [48]:
model.save_weights("new.h5")

In [46]:
predictions=np.argmax(model.predict(test_x),axis=1)
print(predictions.shape)
predictions=label_to_string(predictions)
predictions=np.asarray(predictions)
print(predictions.shape)

(100000,)
(100000,)


In [47]:
df=pd.DataFrame(data=predictions,columns=['CATEGORY'])
df.index.name='ID'
df.to_csv(path_or_buf="ignore/CNN_sub.csv",sep=',')

In [16]:

# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")
 
# # later...
 
# # load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
 
# # evaluate loaded model on test data
# loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
# score = loaded_model.evaluate(X, Y, verbose=0)
# print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

In [8]:
model.load_weights("new.h5")